<a href="https://colab.research.google.com/github/awanshrestha/fake-news-classification/blob/main/fake_news_classification_with_rf_awan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
true_news = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/True.csv")
true_news["target"] = 1
print(true_news.shape)
true_news.head(10)

(21417, 5)


,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017",1
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017",1
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017",1
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017",1
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017",1


In [6]:
fake_news = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Fake.csv")
fake_news["target"] = 0
print(fake_news.shape)
fake_news.head(10)

(23481, 5)


,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [7]:
true_news = true_news[["text", "target"]].copy()
true_news.head(10)

,text,target
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
5,"WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",1
6,"WEST PALM BEACH, Fla (Reuters) - President Don...",1
7,The following statements were posted to the ve...,1
8,The following statements were posted to the ve...,1
9,WASHINGTON (Reuters) - Alabama Secretary of St...,1


In [8]:
fake_news = fake_news[["text", "target"]].copy()
fake_news.head(10)

,text,target
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
5,The number of cases of cops brutalizing and ki...,0
6,Donald Trump spent a good portion of his day a...,0
7,In the wake of yet another court decision that...,0
8,Many people have raised the alarm regarding th...,0
9,Just when you might have thought we d get a br...,0


In [9]:
all_news = pd.concat([true_news, fake_news])
print(all_news.shape)
all_news.head(10)

(44898, 2)


,text,target
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
5,"WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",1
6,"WEST PALM BEACH, Fla (Reuters) - President Don...",1
7,The following statements were posted to the ve...,1
8,The following statements were posted to the ve...,1
9,WASHINGTON (Reuters) - Alabama Secretary of St...,1


In [10]:
all_news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44898 non-null  object
 1   target  44898 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [11]:
x_data = all_news["text"].tolist()
y_data = all_news["target"].tolist()

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_data)
x_data = tokenizer.texts_to_sequences(x_data)

In [22]:
print(x_data.shape)

(44898, 8375)


In [13]:
x_data = pad_sequences(x_data)
print("shape of data: ", x_data.shape)

shape of data:  (44898, 8375)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_leaf=10, min_samples_split=2)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=10, n_estimators=200)

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predict_result = clf.predict(X_test)
print("Accuracy:", accuracy_score(predict_result, y_test))
print(classification_report(predict_result, y_test))

Accuracy: 0.689977728285078
              precision    recall  f1-score   support

           0       0.85      0.66      0.74      6012
           1       0.52      0.76      0.62      2968

    accuracy                           0.69      8980
   macro avg       0.68      0.71      0.68      8980
weighted avg       0.74      0.69      0.70      8980



In [25]:
def predict(text):
    data = tokenizer.texts_to_sequences(text)
    data = pad_sequences(data, maxlen=8375)
    clf.predict(data)
    if clf.predict(data)[0] == 1:
        print('This is fake news.')
    else:
        print('This is real news.')

In [26]:
corpus1 = '''One of Donald Trump s favorite punching bags is CNN. He even once tweeted a GIF image of himself punching a person with a CNN logo superimposed over the head 
            indicating that he d like to enact violence against CNN s reporters. Then there was the time he tweeted the  Trump Train roaring over  CNN.  Now, he s back at it 
            this time suggesting that  fake  CNN should be the ones representing America to the world, and that they are doing a bad job. Here is that tweet:.@FoxNews 
            is MUCH more important in the United States than CNN, but outside of the U.S., CNN International is still a major source of (Fake) news, and they represent
            our Nation to the WORLD very poorly. The outside world does not see the truth from them!  Donald J. Trump (@realDonaldTrump) November 25, 2017Of course, 
            it is beneath the dignity of most people to respond to a moronic buffoon like Trump under normal circumstances. However, he is currently squatting in the 
            White House, and has his tiny orange hands on the levers of power   not to mention the nuclear codes   so they have to stoop to a Trumpian level when 
            personally attacked. However, being, well, you know, FIT to be doing the job they are doing, the good folks at CNN Communications fired back at Trump, and 
            their response is nothing short of perfect:It's not CNN's job to represent the U.S to the world. That's yours. Our job is to report the news. #FactsFirst   
            CNN Communications (@CNNPR) November 25, 2017BOOM! Couldn t have asked for a sicker burn than this. And they are right of course   especially the part about
            #FactsFirst. Trump has a problem with the truth, as we all well know. That s what makes what the CNN Communications people replied so fabulous. 
            It is the ultimate truth   something the likes of the pathological orange liar that is Donald Trump knows nothing about.Featured image via  
            Andrew Burton/Getty Images '''



corpus2 = '''WASHINGTON (Reuters) - The No. 2 Republican in the U.S. House of Representatives said on Tuesday the House would likely need to vote again on tax
            legislation on Wednesday morning given that Democrats in the Senate were likely to prevail on a procedural objection. House Majority Leader Kevin McCarthy 
            advised House lawmakers that Senate Democrats were likely to object that the legislation fails to comply with the so-called Byrd rule and were likely to be
            upheld, necessitating a second House vote. “As such, members are further advised that an additional procedural vote on the Motion to Concur is expected 
                tomorrow morning, which will clear the bill for President Trump’s signature,” McCarthy said in a notice to House lawmakers. '''

predict(corpus1)
predict(corpus2)

This is real news.
This is real news.
